# Suggestion for API

In [ ]:
import numpy as np
from scipy import stats

import elfi
from elfi import methods, simulators

def autocov(lag, x):
    """
    Normalized autocovariance (i.e. autocorrelation) assuming a (weak) stationary process.
    Assuming univariate stochastic process with realizations in rows
    """
    mu = np.mean(x, axis=1, keepdims=True)
    var = np.var(x, axis=1, keepdims=True, ddof=1)
    # Autocovariance
    C = np.mean(x[:,lag:] * x[:,:-lag], axis=1, keepdims=True) - mu**2
    # Normalize
    tau = C / var
    return tau

The graph dependencies are described through decorators. Using scipy distributions as priors directly almost works. They get converted to elfi distributions correctly but there is no way to reference to the generated node in the BOLFI method.

In [ ]:
# t1 = stats.uniform(0, 2)
# almost works but there is no smart way to reference to the generated elfi distribution at the moment

t1 = elfi.Prior('t1', 'uniform',0, 1)
t2 = elfi.Prior('t2', 'uniform', 0, 1)

MA2 = simulators.MA2(n_obs=1000, t1_act=0.6, t2_act=0.2, t1=t1, t2=t2)

@elfi.summary(MA2)
def S1(x):
    """Compute a summary statistic from the results of the MA1 node."""
    return autocov(lag=1, x=x)


@elfi.summary(MA2)
def S2(x):
    """compute a summary from MA2."""
    return autocov(lag=2, x=x)


@elfi.discrepancy(S1, S2)
def D(x, y):
    """Compute the discrepancy"""
    return np.linalg.norm(np.array(x) - np.array(y), ord=2, axis=0)

elfi.draw_model(D)

In [ ]:
n_sim = 60
n_batch = 4  # find automatically from nr. of cores?

# get rid of explicit dimensions in GPyModel use len(param_nodes)?
gp_model = elfi.GPyModel(input_dim=2, bounds=((0,1),(0,1)), kernel_var=0.3, kernel_scale=0.2, noise_var=0.01)
bolfi = methods.BOLFI(D, [t1, t2], batch_size=n_batch, n_surrogate_samples=n_sim, model=gp_model)
post = bolfi.infer()

In [ ]:
%matplotlib notebook
p = bolfi.plot()

In [ ]:
# Not 2d yet
#p = post.plot()

p = elfi.visualization.plot(lambda x: bolfi.model.gp.predict(x)[0], ((0,1), (0,1)), type='contour')

In [ ]:
p = elfi.visualization.plot(lambda x: bolfi.model.gp.predict(x)[0], ((0,1), (0,1)), type='surface')

In [ ]:
bolfi.display()

# Ideas for improvement

It would be better to be able to use keyword asguments for the nodes to link them to the function arguments. Using just arguments is a bit brittle especially since we are heavily using `super()` calls.

We could define some properties of the domains of the arguments in function annotations (py3 only). This way for example we wouldn't have to pass bounds explicity to each method that may use them. We could also have other information there. Such as discretness, separability from other arguments etc. This would possibly also allow for transformations of 1d outputs so they wouldn't have to be wrapped in `numpy.atleast_2d`.

In [ ]:
class FooDomain:
    def __init__(self, *args, **kwargs):
        pass
    
class BarDomain:
    def __init__(self, *args, **kwargs):
        pass

@elfi.simulator(t1=stats.uniform(0, 1), t2=stats.uniform(0, 1))
def MA2_simulator(t1: FooDomain(bounds=(0, 1)),
                  t2: FooDomain(bounds=(0, 1))) -> BarDomain():
#     generate_data(t1, t2)
    pass